In [8]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from function import lineal, sigmoide, derivate_sigmoide, derivate_lineal,Backpropagation, derivate_error,Gradientdescent
import os
import math

In [9]:
#Inicializamos los pesos de la Capa 1
w = np.random.random(500)
w11 = w[:100]
w12 = w[100:200]
w13 = w[200:300]
w14 = w[300:400]
w15 = w[400:500]
W1=[w11,w12,w13,w14,w15]
#Inicializamos los pesos anteriores
w = np.zeros(500)
w11ant = w[:100]
w12ant = w[100:200]
w13ant = w[200:300]
w14ant = w[300:400]
w15ant = w[400:500]
W1ant = [w11ant,w12ant,w13ant,w14ant,w15ant]

#Inicializamos los pesos de la Capa 2
w = np.random.random(25)
w21 = w[:5]
w22 = w[5:10]
w23 = w[10:15]
w24 = w[15:20]
w25 = w[20:25]
W2 = [w21,w22,w23,w24,w25] 
#Inicializamos los pesos anteriores
w = np.zeros(25)
w21ant = w[:5]
w22ant = w[5:10]
w23ant = w[10:15]
w24ant = w[15:20]
w25ant = w[20:25]
W2ant = [w21ant,w22ant,w23ant,w24ant,w25ant]

#Incializamos los pesos de la Capa 3
w = np.random.random(15)
w31 = w[:5]
w32 = w[5:10]
w33 = w[10:15]
W3 = [w31,w32,w33]
#Inicializamos los pesos anteriores
w = np.zeros(15)
w31ant = w[:5]
w32ant = w[5:10]
w33ant = w[10:15]
W3ant = [w31ant,w32ant,w33ant]
#Conversiones

W1 = np.array(W1)
W2 = np.array(W2)
W3 = np.array(W3)
W = [W1,W2,W3]

W1ant = np.array(W1ant)
W2ant = np.array(W2ant)
W3ant = np.array(W3ant)
Want = [W1ant,W2ant,W3ant]

#Bias
b = random.random()

#Factor de aprendizaje
alfa = np.array(random.random())
beta = np.array(random.random())

#Codificacion de clases
#b = 100
#d = 010
#f = 001

In [10]:
data = (pd.read_csv(os.path.join(os.path.abspath(''),"data","distorsionadas",'100','letras.csv'),sep=';',header=None)).to_numpy()
#Convertimos el dataframe en array numpy
data_train = data[:int(len(data)*0.8)]
data_test = data[int(len(data)*0.8)+1:int(len(data)*0.8)+int(len(data)*0.15)]
data_validation = data[int(len(data)*0.8)+int(len(data)*0.15)+1:99]

In [11]:
#Dividiendo data train en una tupla (entrada,clase)
letras_train = []
for letra in data_train:
    x_test = letra[:100]
    y_test = letra[100:]
    letras_train.append((x_test, y_test))


In [12]:
print(W[2])
for i in range(1):

    #Primer capa
    ys1 =[]
    for j in range(len(W1)):
        #Suma ponderada
        Z = np.dot(letras_train[i][0],W1[j])
        #Funcion de activacion
        ys1.append(lineal(Z+b))
    #print("Primer capa: ", ys1)

    #Segunda capa
    ys2 = []
    for j in range(len(W2)):
        #Suma ponderada
        Z = np.dot(ys1,W2[j])
        #Funcion de activacion
        ys2.append(lineal(Z+b))
    #print("2da capa: ", ys2)
    #Capa de salida
    ys3 = []
    for j in range(len(W3)):
        #Suma ponderada
        Z = np.dot(ys2,W3[j])
        #Funcion de activacion
        ys3.append(sigmoide(Z+b))
    #print("Capa salida: ", ys3)

    #!!-------Backpropagation-----!!
    ys1 = np.array(ys1)
    ys2 = np.array(ys2)
    ys3 = np.array(ys3)
    activations = [ys1,ys2,ys3]
    ye = letras_train[i][1]
    deltas = Backpropagation(ye,activations,W[1:])
    #!!----- Gradient descent------!!
    Want, W = Gradientdescent(deltas,W,Want,alfa,beta)


[[0.76436713 0.90421145 0.49100184 0.20301242 0.38724298]
 [0.59061022 0.43181977 0.36213616 0.6590047  0.06262891]
 [0.36064391 0.74763696 0.46101395 0.14692194 0.01542044]]


In [13]:
print(W[2])
print(W2ant[2])

[[1.18346059 1.38441923 0.79063009 0.37678445 0.64152686]
 [0.80935563 0.5811711  0.48103458 0.90763965 0.05063764]
 [0.60480332 1.16091883 0.74903675 0.29768126 0.1087114 ]]
[0. 0. 0. 0. 0.]


In [ ]:

## PREDICT 
letra = ["b","d","f"]
porcentaje = 0
for i in range(len(1)):
    ys1 =[]
    z1 = []
    for j in range(len(w1)):
                #Suma ponderada
        Z1 = np.dot(letras_train[i][0],w1[j])
                #Funcion de activacion
        ys1.append(lineal(Z1+b1[j]))
        z1.append(Z1)
                
    z1 = np.array(z1)
    ys1 = np.array(ys1)

    #print("Primer capa: ", ys1)
    #Segunda capa
    ys2 = []
    z2 = []
    for j in range(len(w2)):
        #Suma ponderada
        Z2 = np.dot(ys1,w2[j])
        #Funcion de activacion
        ys2.append(f.lineal(Z2+b2[j]))
        z2.append(Z2)
                
    z2 = np.array(z2)
    ys2 = np.array(ys2)

    #print("2da capa: ", ys2)
    #Capa de salida
    ys3 = []
    z3 = []
    for j in range(len(w3)):
        #Suma ponderada
        Z3 = np.dot(ys2,w3[j])
        #Funcion de activacion
        ys3.append(f.sigmoide(Z3+b3[j]))
        z3.append(Z3)
                #print("Capa salida: ", ys3)
                #!!-----Calculo del error-----!!!

    ys3 = np.array(ys3)
    z3 = np.array(z3)

    if(np.argmax(ys3) == np.argmax(letras_train[i][1])):
        porcentaje += 1
    print("Prediccion: {} letra: {} ---- valor real: {}, letra: {}".format( np.argmax(ys3)+1,letra[ np.argmax(ys3)],np.argmax(data_test_predict[i][1])+1,letra[np.argmax(data_test_predict[i][1])] ))
print("Porcentaje predicho: {}%".format((porcentaje/len(letras_train))*100))